In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import bq_helper
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
google_analytics = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="data:google_analytics_sample")

Using Kaggle's public dataset BigQuery integration.


In [3]:
bq_assistant = BigQueryHelper("bigquery-public-data", "google_analytics_sample")

Using Kaggle's public dataset BigQuery integration.


# **Main Query**

In [4]:
#MAIN QUERY
main_query = """SELECT 
                   session_id,
                   channel_grouping,
                   page_url,
                   hit_type,
                   event_action,
                   event_value,
                   hit_start_time,
                   interaction_hit_flag,
                   entrance_flag,
                   exit_flag,
                   product_name,
                   transaction_id,
                   transaction_sales_amount,
                   transaction_currency_code,
                   product_array.productSKU as product_sku,
                   product_array.v2ProductName as product_name,
                   product_array.productQuantity as product_quantity,
                   ROUND(product_array.productPrice/1000000,2) as product_price,
                   ROUND(product_array.productRevenue/1000000,2) as product_revenue
            FROM
            (SELECT date, 
                   fullVisitorId as session_id,
                   channelGrouping as channel_grouping,
                   hits.page.pagePath as page_url,
                   hits.type as hit_type,
                   hits.eventInfo.eventAction as event_action,
                   hits.eventInfo.eventValue as event_value,
                   hits.time as hit_start_time,
                   hits.isInteraction as interaction_hit_flag,
                   hits.isEntrance as entrance_flag,
                   hits.isExit as exit_flag,
                   hits.item.productName as product_name,
                   hits.transaction.transactionID as transaction_id,
                   hits.product as product_array,
                   ROUND(hits.transaction.transactionRevenue/1000000,2) as transaction_sales_amount,
                   hits.transaction.currencyCode as transaction_currency_code
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160801` as GA,
            UNNEST(GA.hits) as hits
            ORDER BY session_id, hit_start_time ASC) as t CROSS JOIN UNNEST(t.product_array) as product_array;"""

main_query = google_analytics.query_to_pandas_safe(main_query)

main_query.head(5)

,session_id,channel_grouping,page_url,hit_type,event_action,event_value,hit_start_time,interaction_hit_flag,entrance_flag,exit_flag,product_name,transaction_id,transaction_sales_amount,transaction_currency_code,product_sku,product_name_1,product_quantity,product_price,product_revenue
0,000722514342430295,Direct,/google+redesign/apparel,PAGE,None,NaN,18101,True,None,None,None,None,NaN,USD,GGOEGHPA002910,Google Trucker Hat,NaN,28.99,NaN
1,000722514342430295,Direct,/google+redesign/apparel,PAGE,None,NaN,18101,True,None,None,None,None,NaN,USD,GGOEYHPA003510,YouTube Trucker Hat,NaN,30.99,NaN
2,000722514342430295,Direct,/google+redesign/apparel,PAGE,None,NaN,18101,True,None,None,None,None,NaN,USD,GGOEGAAX0104,Google Men's 100% Cotton Short Sleeve Hero Tee...,NaN,16.99,NaN
3,000722514342430295,Direct,/google+redesign/apparel,PAGE,None,NaN,18101,True,None,None,None,None,NaN,USD,GGOEGAAX0105,Google Men's 100% Cotton Short Sleeve Hero Tee...,NaN,16.99,NaN
4,000722514342430295,Direct,/google+redesign/apparel,PAGE,None,NaN,18101,True,None,None,None,None,NaN,USD,GGOEGAAX0107,Google Men's 100% Cotton Short Sleeve Hero Tee...,NaN,16.99,NaN


# Metric Calculations

**Conversion Rate**

In [5]:
#Conversion Rate
df = main_query.groupby('session_id')['product_revenue'].sum().reset_index().sort_values(by = 'product_revenue', ascending = False)

total_sessions = len(df)
sessions_w_transactions = len(df.query('product_revenue!=0'))
conversion_pct = round((sessions_w_transactions / total_sessions) * 100,2)

print("conversion: " + str(conversion_pct) + " %")

conversion: 3.83 %


**Bounce Rate**

In [6]:
#Bounce Rate - should be calculated at the page level
#for simplicity let's calculate bounce rate of /home
query = """ SELECT 
                session_id,
                visitStartTime,
                page_url,
                CASE
                    WHEN hitNumber = first_interaction_hit THEN 1 
                    ELSE 0 
                END AS bounces
            FROM
            (SELECT date, 
                   fullVisitorId as session_id,
                   visitStartTime,
                   hits.page.pagePath as page_url,
                   hits.hitNumber,
                   hits.type as hit_type,
                   hits.eventInfo.eventAction as event_action,
                   hits.time as hit_start_time,
                   hits.isEntrance as entrance_flag,
                   hits.isExit as exit_flag,
                   hits.isInteraction, 
                   MIN(IF(hits.IsInteraction IS NOT NULL, hitNumber, 0)) OVER (PARTITION BY fullVisitorId, visitStartTime) as first_interaction_hit
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160801` as GA,
            UNNEST(GA.hits) as hits
            ORDER BY session_id, hit_start_time ASC)"""

response = google_analytics.query_to_pandas_safe(query)

response.head(40)

,session_id,visitStartTime,page_url,bounces
0,000722514342430295,1470093727,/home,1
1,000722514342430295,1470093727,/home,0
2,000722514342430295,1470093727,/google+redesign/apparel,0
3,000722514342430295,1470093727,/asearch.html,0
4,000722514342430295,1470093727,/asearch.html,0
5,0014659935183303341,1470037282,/home,1
6,0015694432801235877,1470043732,/home,1
7,0015694432801235877,1470043732,/google+redesign/electronics,0
8,0015694432801235877,1470043732,/google+redesign/apparel/men++s/men++s+t+shirts,0
9,0015694432801235877,1470043732,/google+redesign/apparel/kid+s/kid+s+infant,0


In [7]:
#Unique pages visited in all sessions combined
query2 = """SELECT hits.page.pagePath
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160802` AS GA, UNNEST(GA.hits) AS hits
            GROUP BY hits.page.pagePath"""

response2 = google_analytics.query_to_pandas_safe(query2)

response2

,pagePath
0,/home
1,/google+redesign/apparel/men++s
2,/google+redesign/bags
3,/google+redesign/drinkware
4,/google+redesign/electronics
...,...
244,/google+redesign/apparel/womens/womens+tshirts...
245,/google+redesign/apparel/women+s+google+perfor...
246,/google+redesign/apparel/women+s+google+short+...
247,/google+redesign/apparel/womens/womens+tshirts...


In [8]:
#number of pageviews
query3 = """SELECT hits.page.pagePath, count(*) as page_views
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160802` AS GA, UNNEST(GA.hits) AS hits
            WHERE hits.type = 'PAGE'
            GROUP BY hits.page.pagePath
            ORDER BY page_views desc
            """
response3 = google_analytics.query_to_pandas(query3)
response3

,pagePath,page_views
0,/home,2664
1,/basket.html,830
2,/google+redesign/office,481
3,/google+redesign/bags,446
4,/google+redesign/electronics,436
...,...,...
244,/google+redesign/apparel/womens/womens+tshirts...,1
245,/google+redesign/apparel/women+s+google+perfor...,1
246,/google+redesign/apparel/women+s+google+short+...,1
247,/google+redesign/apparel/womens/womens+tshirts...,1


In [9]:
#Unique number of unique pageviews --> counts the pageviews by an user only once
query4 = """SELECT pagePath, COUNT(*) as page_views, COUNT(DISTINCT session_id) as unique_page_views
            FROM
            (SELECT hits.page.pagePath, CONCAT(fullVisitorId, CAST(visitStartTime as STRING)) as session_id
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160802` AS GA, UNNEST(GA.hits) AS hits
            WHERE hits.type = 'PAGE')
            GROUP BY pagePath
            ORDER BY page_views desc
            """
response4 = google_analytics.query_to_pandas(query4)
response4

,pagePath,page_views,unique_page_views
0,/home,2664,1798
1,/basket.html,830,198
2,/google+redesign/office,481,342
3,/google+redesign/bags,446,320
4,/google+redesign/electronics,436,329
...,...,...,...
244,/google+redesign/apparel/womens/womens+tshirts...,1,1
245,/google+redesign/apparel/women+s+google+perfor...,1,1
246,/google+redesign/apparel/women+s+google+short+...,1,1
247,/google+redesign/apparel/womens/womens+tshirts...,1,1


In [10]:
#Exits
query5 = """SELECT pagePath, sum(exits) AS exits
            FROM
                (SELECT hits.page.pagePath, 
                CASE 
                    WHEN hits.isExit IS NOT NULL THEN 1
                    ELSE 0
                END AS exits
                FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160802` AS GA, UNNEST(GA.hits) AS hits)
            GROUP BY pagePath
            ORDER BY exits DESC
            """
response5 = google_analytics.query_to_pandas(query5)
response5

,pagePath,exits
0,/home,1022
1,/google+redesign/office,72
2,/google+redesign/apparel/men++s/men++s+t+shirts,71
3,/google+redesign/bags,67
4,/google+redesign/drinkware,57
...,...,...
244,/google+redesign/apparel/men+s+t+shirts/short+...,0
245,/google+redesign/apparel/womens/womens+tshirts...,0
246,/google+redesign/apparel/women+s+google+perfor...,0
247,/google+redesign/apparel/womens/womens+tshirts...,0


In [11]:
#Total Time on Page
query6 = """SELECT fullVisitorId, 
                   visitStartTime,
                   pagePath,
                   hit_time,
                   LEAD(hit_time) OVER (PARTITION BY fullVisitorId, visitStartTime ORDER BY hit_time) AS next_pageview_time
            FROM
            (SELECT fullVisitorId,
                   visitStartTime,
                   hits.page.pagePath,
                   hits.time/1000 AS hit_time
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160802` AS GA, UNNEST(GA.hits) AS hits
            WHERE hits.type = 'PAGE')
            ORDER BY fullVisitorId DESC
            
        """

response6 = google_analytics.query_to_pandas(query6)
response6.head(30)

,fullVisitorId,visitStartTime,pagePath,hit_time,next_pageview_time
0,9993178476401172532,1470164394,/home,0.000,NaN
1,9987572635825356244,1470175580,/google+redesign/office/stickers/home,0.000,NaN
2,9985904536279830835,1470171312,/google+redesign/bags,0.000,5.742
3,9985904536279830835,1470171312,/google+redesign/drinkware,5.742,NaN
4,9970869838545698965,1470202785,/home,0.000,67.164
5,9970869838545698965,1470202785,/home,67.164,NaN
6,9962945586003244765,1470167216,/home,0.000,NaN
7,9959282832735916511,1470175550,/home,0.000,18.816
8,9959282832735916511,1470175550,/home,18.816,104.761
9,9959282832735916511,1470175550,/home,104.761,NaN


In [12]:
#We can create a new column with the timestamp for the last interaction 
#hit by finding the maximum timestamp out of all of the interaction events. 
#Interaction hits are designated by the field hits.isInteraction.

query7 = """SELECT 
                fullVisitorId,
                visitStartTime,
                hits.page.pagePath,
                MAX(IF(hits.isInteraction IS NOT NULL,hits.time,0)) OVER (PARTITION BY fullVisitorId, visitStartTime) as last_interaction
            FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`,
            UNNEST(hits) AS hits
            WHERE hits.type = 'PAGE'
        """

response7 = google_analytics.query_to_pandas(query7)
response7.head(30)

,fullVisitorId,visitStartTime,pagePath,last_interaction
0,0350370813631305535,1470053049,/home,280860
1,0350370813631305535,1470053049,/google+redesign/bags,280860
2,0350370813631305535,1470053049,/google+redesign/bags/quickview,280860
3,0350370813631305535,1470053049,/google+redesign/bags/backpacks/deluge+waterpr...,280860
4,0350370813631305535,1470053049,/google+redesign/lifestyle,280860
5,0350370813631305535,1470053049,/google+redesign/apparel/men++s/men++s+t+shirts,280860
6,0350370813631305535,1470053049,/google+redesign/apparel/men++s/men++s+t+shirt...,280860
7,0350370813631305535,1470053049,/basket.html,280860
8,0350370813631305535,1470053049,/google+redesign/apparel/men++s/men++s+t+shirts,280860
9,0350370813631305535,1470053049,/google+redesign/apparel/men++s/men++s+t+shirt...,280860
